# Proyecto 2
## Data Science, sección 40
## Grupo 1
Javier Alejandro Ovalle Chiquín, 22103  
José Ángel Morales Farfan, 22689  
Ricardo Josué Morales Contreras, 22289  
Karen Daniela Pineda Ventura 231132